<a href="https://www.kaggle.com/code/sogamja/2022-tps-apr-convnet?scriptVersionId=105928568" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Uses skip connection
https://towardsdatascience.com/residual-blocks-building-blocks-of-resnet-fd90ca15d6ec

In [1]:
input_path = '../input/tabular-playground-series-apr-2022/'
output_path = './'

In [2]:
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score

def load_raw_data(train_or_test='train'):
    file_name = f'{input_path}/{train_or_test}.csv'
    df = pd.read_csv(file_name)
    return df

def load_label(train_or_test='train'):
    file_name = input_path + ('train_labels.csv' if train_or_test=='train' else 'sample_submission.csv')
    df = pd.read_csv(file_name)
    return df['state'].values

def competition_metric(y_true, y_score):
    return roc_auc_score(y_true, y_score)

def evaluate(model, X, y):
    return competition_metric(y, model.predict_proba(X)[:, 1])

def submit(arr):
    df = pd.read_csv(f'{input_path}/sample_submission.csv')
    df['state'] = arr
    df.to_csv(f'{output_path}/submission.csv', index=False)

In [3]:
import tensorflow as tf
from tensorflow import keras
from sklearn.base import BaseEstimator, TransformerMixin

class ResNetModel(keras.Model):
    def __init__(self):
        super(ResNetModel, self).__init__()
        self.fns = [
            keras.layers.Conv1D(filters=20, kernel_size=8, padding='same', activation='elu', kernel_regularizer=keras.regularizers.L2(1e-3)), 
            keras.layers.Conv1D(filters=20, kernel_size=8, padding='same', activation='elu', kernel_regularizer=keras.regularizers.L2(1e-3)),
            keras.layers.Conv1D(filters=20, kernel_size=8, padding='same', activation='elu', kernel_regularizer=keras.regularizers.L2(1e-3)),
            keras.layers.AveragePooling1D(2),
            
            keras.layers.Conv1D(filters=20, kernel_size=6, padding='same', activation='elu', kernel_regularizer=keras.regularizers.L2(1e-3)),
            keras.layers.Conv1D(filters=20, kernel_size=6, padding='same', activation='elu', kernel_regularizer=keras.regularizers.L2(1e-3)),
            keras.layers.Conv1D(filters=20, kernel_size=6, padding='same', activation='elu', kernel_regularizer=keras.regularizers.L2(1e-3)),
            keras.layers.AveragePooling1D(2),
            
            keras.layers.Conv1D(filters=20, kernel_size=4, padding='same', activation='elu', kernel_regularizer=keras.regularizers.L2(1e-3)),
            keras.layers.Conv1D(filters=20, kernel_size=4, padding='same', activation='elu', kernel_regularizer=keras.regularizers.L2(1e-3)),
            keras.layers.Conv1D(filters=20, kernel_size=4, padding='same', activation='elu', kernel_regularizer=keras.regularizers.L2(1e-3)),
            keras.layers.AveragePooling1D(3),
            
            keras.layers.GlobalAveragePooling1D(),
            keras.layers.Dense(1, activation='sigmoid')
        ]
        self.bns = [
            keras.layers.BatchNormalization(), 
            keras.layers.BatchNormalization(), 
            
            keras.layers.BatchNormalization(), 
            keras.layers.BatchNormalization(), 
            
            keras.layers.BatchNormalization(), 
            keras.layers.BatchNormalization(), 
            
            keras.layers.BatchNormalization(), 
            keras.layers.BatchNormalization(), 
        ]
        
    def call(self, inputs):
        outputs = inputs
        
        outputs = self.fns[0](outputs)
        res = outputs
        res = self.fns[1](res)
        res = self.fns[2](res)
        outputs += res
        outputs = self.fns[3](outputs)
        
        outputs = self.fns[4](outputs)
        res = outputs
        res = self.fns[5](res)
        res = self.fns[6](res)
        outputs += res
        outputs = self.fns[7](outputs)
        
        outputs = self.fns[8](outputs)
        res = outputs
        res = self.fns[9](res)
        res = self.fns[10](res)
        outputs += res
        outputs = self.fns[11](outputs)
        
        outputs = self.fns[-2](outputs)
        outputs = self.fns[-1](outputs)
        
        return outputs
    
    def predict_proba(self, X):
        return np.concatenate([1-self.predict(X), self.predict(X)], axis=1)

def random_sensor_swap(x, y, random_state=None):
    rng = np.random.default_rng(random_state)
    p_swap = 0.5
    indices = rng.choice(np.arange(x.shape[0]), int(p_swap*x.shape[0]), replace=False)
    x_aug, y_aug = x[indices], y[indices]
    swap_codes = rng.integers(0, 13, (x_aug.shape[0], 2))
    for i in range(x_aug.shape[0]):
        a, b = swap_codes[i]
        x_aug[i, :, [a, b]] = x_aug[i, :, [b, a]]
    x = np.concatenate([x, x_aug], axis=0)
    y = np.concatenate([y, y_aug], axis=0)
    return x, y

def group_splitter(df, nfold=5, random_state=None):
    subject_nums = df['subject'].unique()
    rng = np.random.default_rng(random_state)
    subject_to_setnum = rng.integers(0, nfold, subject_nums.shape[0])
    for i in range(nfold):
        val_subjects = subject_nums[subject_to_setnum == i]
        mask_df_val = df['subject'].isin(val_subjects)
        mask_y_val = mask_df_val.iloc[::60]
        yield mask_df_val, mask_y_val


class DF2arr(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        return X.loc[:, 'sensor_00':'sensor_12'].values.reshape(-1, 60, 13)
    
    
class MyPreprocessor(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        X = normalize(X)
        return X
    
def normalize(x):
    x = x / (np.linalg.norm(x, axis=1, keepdims=True) + 1e-10)
    return x

In [4]:
from sklearn.pipeline import make_pipeline
from sklearn.metrics import classification_report
cv_scores = []

df = load_raw_data('train')
X = DF2arr().transform(df)
y = load_label('train')
subj_nums = df['subject']

preprocessor = make_pipeline(DF2arr(), MyPreprocessor())

keras.backend.clear_session()
tf.random.set_seed(42)
callbacks = [
    keras.callbacks.EarlyStopping(patience=200, restore_best_weights=True)
]

for mask_df_val, mask_y_val in group_splitter(df, nfold=5, random_state=42):
    df_train, df_val = df[~mask_df_val], df[mask_df_val]
    y_train, y_val = y[~mask_y_val], y[mask_y_val]
    for mask_df_v, mask_y_v in group_splitter(df_train, nfold=5, random_state=42):
        df_t, df_v = df_train[~mask_df_v], df_train[mask_df_v]
        y_t, y_v = y_train[~mask_y_v], y_train[mask_y_v]

    X_t = preprocessor.fit_transform(df_t)
    X_v = preprocessor.transform(df_v)
    X_val = preprocessor.transform(df_val)


    model = ResNetModel()
    model.compile(
        loss='binary_crossentropy', 
        metrics=['AUC'],
        optimizer=keras.optimizers.Adam(1e-3))
    model.fit(
        X_t, y_t, 
        batch_size=1024,
        epochs=500, 
        callbacks=callbacks,
        validation_data=(X_v, y_v),
        verbose=0,
    )
    print(evaluate(model, X_val, y_val))
    print(classification_report(y_val, (model.predict(X_val) >= 0.5).astype(int), digits=4 ))
    
    cv_scores.append(evaluate(model, X_val, y_val))
print(f'5-fold CV score: {np.mean(cv_scores):.4f}')

2022-09-18 06:11:59.356012: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.
2022-09-18 06:11:59.572525: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


0.9640761214353601
              precision    recall  f1-score   support

           0     0.9014    0.9032    0.9023      2592
           1     0.9017    0.9000    0.9008      2559

    accuracy                         0.9016      5151
   macro avg     0.9016    0.9016    0.9016      5151
weighted avg     0.9016    0.9016    0.9016      5151

0.9390609822401481
              precision    recall  f1-score   support

           0     0.8626    0.8951    0.8785      2412
           1     0.8793    0.8427    0.8606      2187

    accuracy                         0.8702      4599
   macro avg     0.8709    0.8689    0.8696      4599
weighted avg     0.8705    0.8702    0.8700      4599

0.9630005013028857
              precision    recall  f1-score   support

           0     0.8847    0.8939    0.8892      2789
           1     0.9071    0.8989    0.9030      3215

    accuracy                         0.8966      6004
   macro avg     0.8959    0.8964    0.8961      6004
weighted avg     

In [5]:
df_test_final = load_raw_data('test')

X_train = preprocessor.fit_transform(df_train)
X_val = preprocessor.transform(df_val)
X_test_final = preprocessor.transform(df_test_final)

model.fit(X_train, y_train, 
          epochs=500, 
          batch_size=1024,
          callbacks=callbacks,
          validation_data=(X_val, y_val)
         )
y_pred = model.predict(X_test_final)
submit(y_pred)

Epoch 1/500
21/21 [==============================] - 3s 120ms/step - loss: 0.3089 - auc: 0.9614 - val_loss: 0.3444 - val_auc: 0.9497
Epoch 2/500
21/21 [==============================] - 2s 117ms/step - loss: 0.3081 - auc: 0.9615 - val_loss: 0.3543 - val_auc: 0.9507
Epoch 3/500
21/21 [==============================] - 3s 156ms/step - loss: 0.3072 - auc: 0.9618 - val_loss: 0.3388 - val_auc: 0.9516
Epoch 4/500
21/21 [==============================] - 3s 120ms/step - loss: 0.3055 - auc: 0.9624 - val_loss: 0.3422 - val_auc: 0.9516
Epoch 5/500
21/21 [==============================] - 2s 115ms/step - loss: 0.3049 - auc: 0.9625 - val_loss: 0.3423 - val_auc: 0.9501
Epoch 6/500
21/21 [==============================] - 2s 116ms/step - loss: 0.3048 - auc: 0.9626 - val_loss: 0.3505 - val_auc: 0.9479
Epoch 7/500
21/21 [==============================] - 2s 115ms/step - loss: 0.3069 - auc: 0.9619 - val_loss: 0.3407 - val_auc: 0.9511
Epoch 8/500
21/21 [==============================] - 2s 119ms/step - 